<a href="https://colab.research.google.com/github/GrigoreGeorgeAlexandru/Colab-projects/blob/main/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema canibalilor și misionarilor

Se consideră că avem un număr egal de canibali și misionari (notat cu N) pe malul unui râu. Ei vor să treacă râul cu ajutorul unei bărci cu M locuri. În barcă pot fi mai puțin de M persoane, dar barca nu se conduce singură (pentru a ajunge de pe un mal pe altul trebuie să fie în ea minim un om). Dacă pe unul dintre maluri sau în barcă numărul de canibali este mai mare strict decât numărul de misionari, atunci canibalii o să îi mănânce pe misionari. Permutările între mal și barcă au loc instantaneu, așadar nu ne punem problema ordinii în care coboară sau urcă oamenii în barcă.
Care este o secvență de acțiuni în urma căreia toți oamenii să ajungă pe celălalt mal fără ca misionarii să ajungă prânz pentru canibali?

*Problema clasica este cu N=3 (canibali / misionari) și M=2 (locuri în barcă).*

# Soluții

## 1. Clasa State

---

Definiți o clasă State, care va reprezenta o stare a jocului, cu câmpurile:
* canibali pe malul stâng;
* misionari pe malul stâng;
* poziția bărcii.

Citiți dintr-un **_fișier_** valorile parametrilor N și M și păstrați-le ca elemente statice în clasă. Puteți pune valori default în clasă N=3 și M=2.

Implementați următoarele metode:
* constructorul clasei -> *\_\_init__*
* egalitatea -> *\_\_eq__*
* transformarea în obiect de tip string a informației nodului curent -> *\_\_str__*
* transformarea în obiect de tip string a informației elementelor dintr-o listă -> *\_\_repr__*

_Observații:_
* Informația nodului curent va fi afișată de forma:
```
Stare curentă:
3 misionari, 3 canibali  | 0 misionari, 0 canibali
Barca se află pe malul stâng
```
* Informația elementului dintr-o listă va fi afișată de forma:
```
(3, 3, 0)
```
Unde tuplul reprezintă (<număr misionari pe malul stâng>, <număr canibali pe malul stâng>, <malul pe care se află barca>)

In [ ]:
class State:
  N = 3
  M = 2

  def __init__(self, misionari = N, canibali = N, barca = -1):
    ''' Constructorul clasei State.

    :param misionari: cati misionari sunt pe malul stang
    :param canibali: cati canibali sunt pe malul stang
    :param barca: -1 daca se afla pe malul stang, 1 altfel
    '''
    self.misionari = misionari
    self.canibali = canibali
    self.barca = barca

  def __eq__(self, cls):
     return self.barca == cls.barca and \
     self.misionari == cls.misionari and self.canibali == cls.canibali

  def __str__(self):
    return 'Stare curenta:\n' + \
    f'{str(self.misionari)} misionari, ' + \
    f'{str(self.canibali)} canibali | ' + \
    f'{str(self.N - self.misionari)} misionari, ' + \
    f'{str(self.N - self.canibali)} canibali \n' + \
    f'Barca se afla pe malul {"stang" if self.barca == -1 else "drept"}\n'

  def __repr__(self):
    return ('({} {} {})').format(self.misionari, self.canibali, self.barca)

  def succesori(self):
    ''' Calculeaza lista succesorilor directi ai starii curente.

    :return: lista starilor admisibile
    '''
    succesori = []

    # Calculez cati misionari si canibali sunt pe malul cu barca
    misionari = self.misionari if self.barca == -1 else self.N - self.misionari
    canibali = self.canibali if self.barca == -1 else self.N - self.canibali

    for locuri in range(1, self.M + 1):
      for locuriMisionari in range(misionari + 1):
        if locuriMisionari > locuri:
          continue

        # Verific sa existe cati canibali vreau sa plimb
        locuriCanibali = locuri - locuriMisionari
        if locuriCanibali < 0 or locuriCanibali > canibali:
          continue

        # Verific sa nu fie mancati misionari in barca
        if locuriCanibali > locuriMisionari and locuriMisionari > 0:
          continue

        # Verific sa nu fie mancati misionari pe malul de plecare
        if canibali - locuriCanibali > misionari - locuriMisionari and \
        misionari - locuriMisionari > 0:
          continue

        # Verific sa nu fie mancati misionari pe malul de sosire
        if (self.N - canibali) + locuriCanibali > \
        (self.N - misionari) + locuriMisionari and \
        (self.N - misionari) + locuriMisionari > 0:
          continue

        # Trucuri de notatie:
        stareCurenta = State(
            self.misionari + self.barca * locuriMisionari,
            self.canibali + self.barca * locuriCanibali,
            (-1) * self.barca)

        succesori.append(stareCurenta)

    return succesori

In [ ]:
f = open("input.txt", "r")
State.N = int(f.readline())
State.M = int(f.readline())
f.close()

In [ ]:
state = State(State.N, State.N, -1)
print(state)

Stare curenta:
3 misionari, 3 canibali | 0 misionari, 0 canibali 
Barca se afla pe malul stang



## 2. Clasa Graph

---

Copiați clasa Node.

Modificați următoarele funcții din clasa Graph pe problema curentă:
* scop - verifică dacă toți oamenii au ajuns pe malul drept
* succesori - generează lista de succesori valizi ai stării curente

_Sugestie:_ Pentru a scrie mai puțin, puteți calcula succesorii unei stări în clasa State și apela acea funcție din clasa Graph. Nu uitați să verificați că succesorul nu a mai fost vizitat în drumul dinspre rădăcină!

In [ ]:
import copy


class Node:
  def __init__(self, informatie, parinte = None, succesori = []):
    ''' Constructorul clasei Node.

    :param informatie: informatia nodului curent
    :param parinte: pointer catre parintele nodului curent
    :param succesori: pointer catre lista succesorilor nodului curent
    '''
    self.informatie = informatie
    self.parinte = parinte
    self.succesori = copy.deepcopy(succesori)

  def __str__(self):
    return str(self.informatie)

  def __repr__(self):
    return ('({} {} {})').format(self.informatie.misionari,
                                 self.informatie.canibali,
                                 self.informatie.barca)

  def __eq__(self, cls):
     return self.informatie == cls.informatie

  def vizitat(self):
    ''' Returneaza True daca nodul curent a fost deja vizitat, False altfel. '''
    return len([1 for nod in self.drumRadacina() if nod == self]) > 1 or \
    self.informatie in [nod.informatie for nod in self.parinte.succesori]

  def drumRadacina(self):
    ''' Calculeaza lista nodurilor de la radacina pana la nodul curent. '''
    if self.parinte is None:
      return [self]
    return self.parinte.drumRadacina() + [self]

  def printDrumRadacina(self):
    ''' Returneaza un string cu formatarea drumului de la radacina pana la nodul
    curent.
    '''
    drum = self.drumRadacina()
    afisare = str(drum[0])

    for i in range(1, len(drum)):
      state = drum[i - 1].informatie
      stateCurent = drum[i].informatie

      afisare += f'\nBarca a plecat de pe malul ' + \
      f'{"stang" if state.barca == -1 else "drept"} ' + \
      f'cu {abs(state.misionari - stateCurent.misionari)} misionari ' + \
      f'si {abs(state.canibali - stateCurent.canibali)} canibali\n\n' + \
      str(stateCurent)

    return afisare

In [ ]:
class Graph:
  def __init__(self):
    ''' Constructorul clasei Graf. '''
    self.nodStart = Node(State())
    self.noduriScop = [State(0, 0, 1)]

  def scop(self, informatieNod):
    ''' Primeste o informatie de tip Nod si verifica daca e scop.

    :param informatieNod: informatia nodului de cautat
    :return: True daca nodul e scop, False altfel
    '''
    return informatieNod in self.noduriScop

  def succesori(self, nod):
    ''' Primeste un nod al arborelui de parcurgere si returnează lista
    succesorilor directi ai nodului care nu au fost vizitati pe ramura curenta.

    :param nod: un nod oarecare
    :return: lista succesorilor valizi ai nodului
    '''
    succesori = nod.informatie.succesori()

    for stare in succesori:
      nodCurent = Node(stare, nod)
      if nodCurent.vizitat():
        continue

      nod.succesori.append(nodCurent)

    return nod.succesori

In [ ]:
graf = Graph()
print(graf.nodStart)

Stare curenta:
3 misionari, 3 canibali | 0 misionari, 0 canibali 
Barca se afla pe malul stang



## 3. Clasa Node

---

Creați o funcție *printDrumRadacina()* care returnează drumul până la nodul curent în formatul de mai jos. Rulați problema cu BFS / DFS pentru N=2, M=3 și NSOL=2 (numărul de soluții cerute) și afișați într-un fișier cu calea dată de la tastatură drumul de la rădăcină până la nodul curent (în formatul precizat anterior) și timpul de rulare al algoritmului:

```
Stare curentă:
3 misionari, 3 canibali  | 0 misionari, 0 canibali
Barca se află pe malul stâng

Barca s-a deplasat de pe malul stâng pe malul drept cu 0 misionari și 2 canibali.

Stare curentă:
3 misionari, 1 canibali  | 0 misionari, 2 canibali
Barca se află pe malul drept

...

Stare curentă:
0 misionari, 0 canibali  | 3 misionari, 3 canibali
Barca se află pe malul drept

Timpul de rulare: 0.00021839141845703125 secunde.
```


In [ ]:
def bfs(graf, n = 2):
  ''' Implementarea tehnicii de cautare in latime (BFS) cu n solutii.

  :param graf: graful pe care rulam algoritmul
  :param n: numarul de solutii cautate
  :return: primele n solutii identificate
  '''
  coada = [graf.nodStart]
  while len(coada) > 0:
    nodCurent = coada.pop(0)

    if graf.scop(nodCurent.informatie):
      return nodCurent

    succesori = graf.succesori(nodCurent)
    coada += copy.deepcopy(succesori)

  return None

In [ ]:
graf = Graph()
bfs(graf)

(0 0 1)

In [ ]:
import time

start_time = time.time()
graf = Graph()
nodFinal = bfs(graf)
end_time = time.time()

drum = nodFinal.printDrumRadacina()
output = input("Calea fișierului în care vom scrie soluția: ")
f = open(output, "w")

if drum:
  f.write(drum)
else:
  f.write("Nu exista solutie.")

f.write(f"\nTimpul de rulare: {end_time - start_time} secunde.")
f.close()

Calea fișierului în care vom scrie soluția: output.txt


## 4. Problemă alternativă

---

Modificați problema astfel încât în starea inițială avem un număr NC de misionari pe malul stâng și un alt număr NM de canibali pe malul drept, iar toată lumea își dorește să traverseze râul.

Canibalii pot mânca misionarii doar pe malul drept sau în barcă (malul stâng este foarte aproape de tabăra misionarilor în care au fost plasate mai mult de NC gărzi care se asigură că totul este în regulă).

Barca se află inițial pe malul stâng (al misionarilor) întrucât aceștia sunt singurii care știu să o conducă.

**Atenție:** asta înseamnă că în barcă trebuie să fie mereu minim un misionar.